# PACKAGES

In [3]:
import pandas as pd
import contextily as cx
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point
import numpy as np
from scipy.spatial.distance import cdist
from shapely.ops import cascaded_union
from shapely.ops import unary_union
import geovoronoi as gv
import geovoronoi.plotting as gvplt
import seaborn as sns
from shapely.geometry import Polygon
from sklearn.neighbors import BallTree

# DATA COLLECTION

In [4]:
masterlist_filename = 'landbank.json'

In [5]:
def nearby(ind, dist, df, branch_index):
    listing = ind[j]
    for indx, i in enumerate(listing):
        prnt_dist = dist[j][indx]*earth_radius
        print('%.0fm'%prnt_dist, df.at[i, 'type_id'], 
          df.at[i, 'address'], df.at[i, 'latitude'], df.at[i, 'longitude'], i, df.at[i, 'id'])
    return 0

In [6]:
df = pd.read_json(masterlist_filename)
df = df.T
df.sort_index(inplace=True)
df.at[13, 'latitude'] = '15.308691'
df.at[460, 'latitude'] = '14.6507700'
df.at[460, 'longitude'] = '120.9721620'
df.at[610, 'latitude'] = '14.6492420'
df.at[610, 'longitude'] = '121.0692769'
df.at[5551, 'latitude'] = '15.5328227'
df.at[5551, 'longitude'] = '119.9570529'
df.drop(df[df.status_id == 0].index, inplace=True)
df.reset_index(inplace=True)

In [7]:
known = [477, 0, 2316, 168, 55]
type_id_label = {1: 'branch and branch-lite', 2: 'LEAF', 3: 'atm', 4: 'cash-deposit machines', 5: 'lending center'}
#411Branches and 66 branch-lite units, 55 Lending Centers and 2,316 ATMs and 168 CDMs 
for i in range(1,6):
    print(i, df[df.type_id ==i].shape, known[i-1])

1 (449, 20) 477
2 (47, 20) 0
3 (2150, 20) 2316
4 (120, 20) 168
5 (57, 20) 55


In [8]:
print(df.shape)
df.tail(5)

(2823, 20)


,index,id,fid,maker_id,checker_id,type_id,region_id,province_id,city_id,status_id,name,address,email,number,fax,longitude,latitude,remarks,created_at,updated_at
2818,5542,5542,0,28,9,1,7,37,682,1,Barotac Viejo (Iloilo) Agri Hub,"Barotac Viejo Trade Center, Zulueta Drive, Bar...",AGRI_BVIEJO@mail.landbank.com,(033) 337-3632,None,122.8497443,11.0392085,None,2021-02-26 14:48:01,2021-03-05 09:13:38
2819,5546,5546,0,28,9,1,7,38,1039,1,Himamaylan Branch Lite,"GM Building, Rizal Street, Barangay 2, Poblaci...",BR_HIMAMAYLAN@mail.landbank.com,(034) 388-3777,None,122.9665772,10.2054372,None,2021-02-26 15:17:50,2021-03-05 09:35:47
2820,5549,5549,0,28,9,1,11,55,984,1,Tangub (Misamis Occidental) Branch,"2nd North corner Magsaysay Street, Barangay II...",BR_TANGUB@mail.landbank.com,(088) 531-3113,None,123.7500494,8.0635912,None,2021-02-26 15:37:21,2021-03-05 09:39:18
2821,5551,5551,0,28,0,1,3,16,1552,2,Masinloc (Zambales) Branch,"National Highway, Barangay Inhobol, Masinloc, ...",BR_MASINLOC@mail.landbank.com,(0966) 239-5481,None,119.9570529,15.5328227,None,2021-03-05 15:00:45,2021-03-05 15:00:45
2822,5552,5552,0,28,0,1,8,39,269,2,Tubigon (Bohol) Branch,"B. Cabangbang Avenue, Barangay Potohan, Tubigo...",BR_TUBIGON@mail.landbank.com,(038) 237-2658,None,123.9484996,9.9360606,None,2021-03-05 15:11:42,2021-03-05 15:11:42


In [9]:
df[df.type_id == 2].id
df[df.id == 5508]
df[df.city_id == 774]

,index,id,fid,maker_id,checker_id,type_id,region_id,province_id,city_id,status_id,name,address,email,number,fax,longitude,latitude,remarks,created_at,updated_at
2017,2181,2181,0,50,68,4,1,3,774,1,CDM LEAF Pugo,"'Poblacion West, Pugo, La Union",N/A,N/A,N/A,120.4677679,16.3123097,None,2019-09-24 16:39:51,2019-09-25 10:29:46
2188,2683,2683,0,50,68,3,1,3,774,1,LEAF Pugo,"Poblacion West, Pugo, La Union",None,N/A,N/A,120.4656775,16.3203602,None,2019-09-25 14:48:33,2019-09-27 16:35:41
2800,5508,5508,0,28,9,2,1,3,774,1,Pugo (La Union) LEAF,"Poblacion West, Pugo, La Union",leaf_pugo@mail.landbank.com,(0917) (0917)-503-7581,None,120.4764692,16.3195419,None,2019-10-03 17:08:19,2019-10-03 17:16:06


In [10]:
print(len(df['province_id'].unique()), len(df['city_id'].unique()))

81 692


In [11]:
long = df['longitude'].astype(float)
lat = df['latitude'].astype(float)
geometry = [Point(xy) for xy in zip(long, lat)]
geo_df = gpd.GeoDataFrame(geometry = geometry)
geo_df = geo_df.set_crs(epsg=4326)

#geo_df.crs = {'init':"epsg:4326"}

In [12]:
# ax = geo_df.plot(color="red", markersize=0.2, figsize=(20, 20))
# #cx.add_basemap(ax, crs=geo_df.crs.to_string())
# # plot the marker
# cx.add_basemap(ax, crs=geo_df.crs.to_string(), source=cx.providers.OpenStreetMap.Mapnik)

In [13]:
# nightlights = cx.providers.NASAGIBS.ViirsEarthAtNight2012
# philippines = cx.Place("Philippines", source=nightlights)
# philippines.plot()

In [14]:
df[df['city_id'] == 17] #14 QC

,index,id,fid,maker_id,checker_id,type_id,region_id,province_id,city_id,status_id,name,address,email,number,fax,longitude,latitude,remarks,created_at,updated_at
41,43,43,0,28,9,1,16,77,17,1,Valenzuela Branch,ARCA North Corporate Center Building #150 F. D...,BR_VLNZUELA@mail.landbank.com,(02) 8292-1971;8292-3688;8292-0114,(02) 8292-1478,120.98321758320519,14.701266365738643,None,2019-06-18 17:04:37,2021-03-05 09:11:25
415,417,417,0,50,68,3,16,77,17,1,Valenzuela Medical Center,"Padrigal Street, Karuhatan, Valenzuela City",N/A,N/A,N/A,120.8990212,14.692362,None,2019-06-20 13:29:18,2019-09-27 13:42:47
417,419,419,0,50,68,3,16,77,17,1,Valenzuela LGU 2,"Valenzuela LGU, Poblacion, Valenzuela City",N/A,N/A,N/A,120.9605635,14.689741,None,2019-06-20 13:30:38,2019-09-27 13:42:42
418,420,420,0,50,68,3,16,77,17,1,Valenzuela LGU 1,"Valenzuela LGU, Poblacion, Valenzuela City",N/A,N/A,N/A,120.9605635,14.689741,None,2019-06-20 13:31:32,2019-09-27 13:41:56
420,422,422,0,50,68,3,16,77,17,1,Valenzuela,"GBT Bldg. Oreta Subd., Maysan Road corner G. T...",N/A,N/A,N/A,120.969059,14.692362,None,2019-06-20 13:33:46,2019-09-27 13:41:50
1950,2110,2110,0,50,68,3,16,77,17,1,Brgy. Punturin,Valenzuela LGU's 3S Center 150 P. Faustino St....,None,N/A,N/A,120.9878149,14.7410312,None,2019-09-24 15:09:18,2019-09-27 13:42:01
1951,2111,2111,0,50,68,3,16,77,17,1,3S Center Marulas,"3S Center, Brgy. Marulas, Valenzuela City",None,N/A,N/A,120.9806544,14.6739805,None,2019-09-24 15:10:28,2019-09-27 13:42:22
1952,2112,2112,0,50,68,3,16,77,17,1,Brgy. Ugong,"3S Center Brgy Ugong,GF Que Grande St. Ugong, ...",None,N/A,N/A,121.0069227,14.6941205,None,2019-09-24 15:11:12,2019-09-27 13:42:27
1953,2113,2113,0,50,68,3,16,77,17,1,Brgy. GT De Leon,"3S Center M. Delos Reyes, Brgy Gen. T De Leon,...",None,N/A,N/A,120.9887583,14.6853377,None,2019-09-24 15:12:16,2019-09-27 13:42:33
1954,2114,2114,0,50,68,3,16,77,17,1,Finance Center Valenzuela LGU,"Mc Arthur Highway, Malinta, Valenzuela",None,N/A,N/A,120.9668703,14.6923672,None,2019-09-24 15:13:57,2019-09-27 13:42:08


In [15]:
df[df['id'] == 610] #610]

,index,id,fid,maker_id,checker_id,type_id,region_id,province_id,city_id,status_id,name,address,email,number,fax,longitude,latitude,remarks,created_at,updated_at
595,610,610,0,50,68,3,16,77,14,1,UP Diliman College of Arts & Sciences,"College of Arts and Sciences, UP Diliman Campu...",N/A,N/A,N/A,121.0692769,14.6492420,None,2019-06-20 16:50:21,2019-09-27 11:33:05


In [16]:
df[pd.to_datetime(df['updated_at']).apply(lambda x: x.year)==2021]

,index,id,fid,maker_id,checker_id,type_id,region_id,province_id,city_id,status_id,name,address,email,number,fax,longitude,latitude,remarks,created_at,updated_at
41,43,43,0,28,9,1,16,77,17,1,Valenzuela Branch,ARCA North Corporate Center Building #150 F. D...,BR_VLNZUELA@mail.landbank.com,(02) 8292-1971;8292-3688;8292-0114,(02) 8292-1478,120.98321758320519,14.701266365738643,None,2019-06-18 17:04:37,2021-03-05 09:11:25
281,283,283,0,28,9,1,15,73,944,1,Parang Branch,"Cor. Mangga and Durian Streets, Poblacion 1, ...",BR_PARANG@mail.landbank.com,(064)\t425-0035 / 425-0160,425-0035,124.2268932,7.38543,None,2019-06-19 16:28:45,2021-03-05 09:45:45
295,297,297,0,28,9,1,13,63,1415,1,Gen. Santos (Gaisano) Branch,"G/F, Vensu Building, National Highway, General...",BR_GNSNGAISANO@mail.landbank.com,(083)\t887-4896 / 552-6075 / 302-6490,552-6075 / 302-6490,125.1842295,6.1160405,None,2019-06-19 16:52:04,2021-03-05 09:41:07
349,351,351,0,28,9,1,16,77,12,1,Ortigas Center-Pearl Drive Branch,"Ground Floor, Tycoon Center Bldg., Pearl Drive...",BR_ORTIGASCTR@mail.landbank.com,(02) 8584-9694,(02) 8584-9692,121.0608533,14.5814685,None,2019-06-20 11:09:06,2021-03-05 09:05:08
1386,1450,1450,0,27,8,5,2,7,717,1,Southern Isabela LC,"2nd Floor Isabela Trade Center San Fermin, Cau...",lbpisabelalc@yahoo.com,"(078) 652 1859, 09088996144",(078) 652-0281,121.767341,16.937539,None,2019-06-24 10:22:09,2021-03-23 10:13:56
1396,1460,1460,0,27,8,5,6,29,380,1,Camarines Sur LC,"2/F LBP Building, Panganiban Drive cor. Magsay...",lbp_nagalc@yahoo.com,(054) 473-3264 / (054) 473-2047,(054) 473-7924,123.19981439999992,13.6278548,None,2019-06-24 11:03:15,2021-03-23 10:13:49
2803,5523,5523,0,27,8,5,1,1,635,1,Ilocos Norte LC,"Venvi IT Park, Ground Floor, Accenture Bldg.Va...",ilocosnortelc@gmail.com,(077) 604-0455,(077)604-0422,18.1807908,120.5917434,None,2021-01-14 13:46:46,2021-03-23 10:14:10
2804,5524,5524,0,27,8,5,2,7,717,1,Northern Isabela LC,"2nd Floor Isabela Trade Center San Fermin, Cau...",southern.isabelalc@yahoo.com,(078) 652 0281,None,121.767341,16.937539,None,2021-01-15 09:40:49,2021-03-23 10:14:03
2805,5527,5527,0,27,8,5,7,34,116,1,Antique LC,"T.A. Fornier st., San Jose, Antique",None,(036) 540 - 9556,None,10.7451364,121.9374064,None,2021-01-15 09:49:37,2021-03-23 10:13:40
2806,5528,5528,0,27,8,5,9,45,876,1,Southern Leyte LC,"2nd Floor, ABC Building, Apo Street, Ormoc City",southernleytelc@yahoo.com,(053) 561 - 3923,None,11.0122532,124.6041615,None,2021-01-15 10:11:22,2021-03-23 10:13:34


In [17]:

earth_radius = 6371000 # in m
test_radius = 50   # in m 

long = np.pi/180*df['longitude'].astype(float)
lat = np.pi/180*df['latitude'].astype(float)
X = [xy for xy in zip(long, lat)]
tree = BallTree(X, metric='haversine')              
dist, ind = tree.query(X, k=20)                
#dist, ind = tree.query_radius(X, r=test_radius/earth_radius, return_distance=True)                

In [18]:
df.head()

,index,id,fid,maker_id,checker_id,type_id,region_id,province_id,city_id,status_id,name,address,email,number,fax,longitude,latitude,remarks,created_at,updated_at
0,1,1,0,0,0,1,3,14,1231,1,"LANDBANK Easy Access Facility (LEAF) Sta. Ana,...","""Stall B, IC Pelayo Bldg., Villa Luisa Sta. Lu...",leaf_staana@mail.landbank.com,(045) 409-0817; (0923) 874-6568,409-0817,120.7687307,15.0940482,None,2019-06-18 15:22:00,2019-06-20 10:24:54
1,3,3,0,0,0,1,3,12,317,1,Sta. Maria Branch,"Formix Bldg., Fortunato Halili Ave. Bagbaguin ...",BR_STAMARIA@mail.landbank.com,(044) 815-4117; 641-2700,228-2577,120.9478396,14.815198,None,2019-06-18 15:28:02,2019-06-20 10:25:16
2,4,4,0,0,0,1,3,13,1118,1,Palayan EO,"Singalat, Palayan City, Nueva Ecija",None,N/A,N/A,121.0978483,15.5619806,None,2019-06-18 15:30:30,2019-06-18 15:30:30
3,5,5,0,0,0,1,3,14,1214,1,West San Fernando (Pampanga) Branch,"Regional Government Center, Maimpis, City of S...",lbp_westsanfernandopampanga@yahoo.com.ph,(045) 455-2391,455-2375,120.7001737,15.0176241,None,2019-06-18 15:35:01,2019-06-20 10:25:39
4,6,6,0,0,0,1,16,77,14,1,Camp Aguinaldo Branch,"AFP Gen. Insurance Corporation Bldg., Bonny Se...",BR_AGUINLDO@mail.landbank.com,(02) 913-4364; 913-4365,911-2590,121.055579,14.610992,None,2019-06-18 15:36:09,2019-06-20 14:07:20


In [19]:
df.to_csv('datapoints.csv')